# HW 5: COMET-ATOMIC Schema

In this homework, you will create your own schema to represent the state of a story world as it goes through the story line by line.
A **schema** is a structured reprensentation made to hold facts or a plan, which in this case, can be used to track change over time.

**The purpose of this homework is to test your understanding of schemas and get hands-on experience with a state-of-the-art tool in commonsense reasoning.**


## Your Task
You will be creating a schema using ATOMIC to track the state of a fictional world. For each sentence of the story, you will parse it (provided), call COMET (provided, but what you input is up to you), create preconditions to determine if a sentence can be added (TODO), and create effects to use to update your schema (TODO).

Let's teach your agent some basic information about the world!

------------------------

Formally, the task is:

Given an input sentence at time *t* (*In_t*), produce a schema *S_t*. Do this for each sentence in the story.

For example, using VerbNet:

| Timestep | Input | Schema |
|--------|-------|--------|
| 1 | Bethany picks up the sword. | `Bethany: has_possesion(sword)` |
| 2 | Bethany throws the sword. | `Bethany: !has_possesion(sword)` |

*In_1* - "Bethany picks up the sword."

would produce

*S_1* - `Bethany: has_possesion(sword)`

But then if the next sentence is:

*In_2* - "Bethany throws the sword."

The state would be updated to

*S_2* - `Bethany: !has_possesion(sword)`

-----------------------------
Your resulting system will be sort of like this simplified diagram (the parser is provided for you):
![Given a sentence_t and the knowledge representation from your knowledge database of choice, produce schemas (via some schema processor you create)](https://interactive-fiction-class.org/homeworks/schemas/schemas.png)

There is some knowledge about the story world, and you are using a schema to feed this information into bite-sized chunks so that your agent (and you) can understand it.
You will then have a processing step on the schema where you update it as you get more information as the story progresses.


To reiterate, you will:
1. Get to know [COMET-ATOMIC-2020](https://aaai.org/ojs/index.php/AAAI/article/view/4160). (Alternate link in case AAAI is down: https://arxiv.org/pdf/2010.05953.pdf)
2. Make a __schema manipulator__ (not a real term, but I think it sounds cool), which will take in knowledge from ATOMIC and spit out your schema. Skeleton code is provided for you, but you are welcome to change things so that it makes more sense to you. This involves two steps:

  a. Validate: Generate preconditions to determine if an event can be added.

  b. Update: Generate effects to update your world state.  



# What is ATOMIC?

ATOMIC is a commonsense knowledge graph with some social inferences, among other things.

```
@inproceedings{sap2019atomic,
   title={ATOMIC: An Atlas of Machine Commonsense for If-Then Reasoning},
   author={Sap, Maarten and LeBras, Ronan and Allaway, Emily and Bhagavatula, Chandra and Lourie, Nicholas and Rashkin, Hannah and Roof, Brendan and Smith, Noah A and Choi, Yejin},
   year={2019},
   booktitle={AAAI},
   url={https://aaai.org/ojs/index.php/AAAI/article/view/4160}
}
```

It contains the following inferences about people and events:

* Because PersonX wanted (xIntent)
* Before, PersonX needed (xNeed, HasPrerequisite)
* PersonX is seen as (xAttr)
* As a result, PersonX feels (xReact)
* As a result, PersonX wants (xWant)
* As a result, PersonX reasons (xReason)
* PersonX then (xEffect)
* As a result, others feel (oReact)
* As a result, others want (oWant)
* Others then (oEffect)
* Happens before (isBefore)
* Happens after (isAfter)
* Is hindered by (HinderedBy)
* Causes (Causes)

and inferences about entities:
* Is located at (AtLocation, LocatedNear, LocationOfAction)
* Is made up of (MadeUpOf, PartOf. NotMadeOf)
* Is used to (UsedFor, ObjectUse)
* Has the property (HasProperty, NotHasProperty)
* Is capable of (CapableOf, NotCapableOf)
* Desires (Desires, NotDesires)


Among other things (full list is in code block 41).
Or you can play with it online here: https://mosaickg.apps.allenai.org/kg_atomic2020
https://mosaickg.apps.allenai.org/model_comet2020_people_events
https://mosaickg.apps.allenai.org/model_comet2020_entities


[COMET](https://aclanthology.org/P19-1470) is the model that is trained on ATOMIC.

#Setup

## Get COMET and Install packages

Repo: https://github.com/allenai/comet-atomic-2020/

In [1]:
# Clone the repo
%%capture
!git clone https://github.com/allenai/comet-atomic-2020.git

In [2]:
# Enter the directory
import os
os.chdir('comet-atomic-2020')

In [3]:
# Install ATOMIC's dependencies
%%capture
!pip install rouge_score
!pip install transformers
!pip install -r requirements.txt

### Also, Stanford's Stanza Parser

In order to process the input sentences, you will need to do some parsing. Here, we have setup Stanford's English language NER (Named Entity Recognition) and constituency parser using [Stanza](https://stanfordnlp.github.io/stanza/index.html). You're also welcome to use any of the other parsers they provide (or even switch to a completely different type of parser that you like better).

In [4]:
%%capture
!python -m pip install stanza

import stanza
stanza.download('en')

**You might need to restart the runtime now.**


In [5]:
# Example code to run Stanza

import stanza #here's the re-import for when your runtime is restarted
import json
nlp = stanza.Pipeline('en', processors='tokenize, ner, mwt') #lemma, depparse, constituency, pos
parse = nlp("Aurora submitted her resignation to Facebook.")
y = json.loads(str(parse))
y

[[{'end_char': 6, 'id': 1, 'ner': 'S-ORG', 'start_char': 0, 'text': 'Aurora'},
  {'end_char': 16, 'id': 2, 'ner': 'O', 'start_char': 7, 'text': 'submitted'},
  {'end_char': 20, 'id': 3, 'ner': 'O', 'start_char': 17, 'text': 'her'},
  {'end_char': 32,
   'id': 4,
   'ner': 'O',
   'start_char': 21,
   'text': 'resignation'},
  {'end_char': 35, 'id': 5, 'ner': 'O', 'start_char': 33, 'text': 'to'},
  {'end_char': 44,
   'id': 6,
   'ner': 'S-ORG',
   'start_char': 36,
   'text': 'Facebook'},
  {'end_char': 45, 'id': 7, 'ner': 'O', 'start_char': 44, 'text': '.'}]]

## COMET-ATOMIC-2020 (BART) Setup

In [6]:
# Download the model
%%capture
!bash models/comet_atomic2020_bart/download_model.sh

In [7]:
%%capture
!pip install gitpython

In [8]:
%%capture
!pip install sacrebleu

**Tip: Take note of the `all_relations` dictionary below! You will need it later on.**

In [9]:
# Load the model
# copied from models/comet_atomic2020_bart/generation_example.py

import json
import torch
import argparse
from tqdm import tqdm
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from models.comet_atomic2020_bart.utils import calculate_rouge, use_task_specific_params, calculate_bleu_score, trim_batch


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


class Comet:
    def __init__(self, model_path):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        task = "summarization"
        use_task_specific_params(self.model, task)
        self.batch_size = 1
        self.decoder_start_token_id = None

    def generate(
            self, 
            queries,
            decode_method="beam", 
            num_generate=5, 
            ):

        with torch.no_grad():
            examples = queries

            decs = []
            for batch in list(chunks(examples, self.batch_size)):

                batch = self.tokenizer(batch, return_tensors="pt", truncation=True, padding="max_length").to(self.device)
                input_ids, attention_mask = trim_batch(**batch, pad_token_id=self.tokenizer.pad_token_id)

                summaries = self.model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    decoder_start_token_id=self.decoder_start_token_id,
                    num_beams=num_generate,
                    num_return_sequences=num_generate,
                    )

                dec = self.tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=False)
                decs.append(dec)

            return decs


all_relations = [
    "AtLocation",
    "CapableOf",
    "Causes",
    "CausesDesire",
    "CreatedBy",
    "DefinedAs",
    "DesireOf",
    "Desires",
    "HasA",
    "HasFirstSubevent",
    "HasLastSubevent",
    "HasPainCharacter",
    "HasPainIntensity",
    "HasPrerequisite",
    "HasProperty",
    "HasSubEvent",
    "HasSubevent",
    "HinderedBy",
    "InheritsFrom",
    "InstanceOf",
    "IsA",
    "LocatedNear",
    "LocationOfAction",
    "MadeOf",
    "MadeUpOf",
    "MotivatedByGoal",
    "NotCapableOf",
    "NotDesires",
    "NotHasA",
    "NotHasProperty",
    "NotIsA",
    "NotMadeOf",
    "ObjectUse",
    "PartOf",
    "ReceivesAction",
    "RelatedTo",
    "SymbolOf",
    "UsedFor",
    "isAfter",
    "isBefore",
    "isFilledBy",
    "oEffect",
    "oReact",
    "oWant",
    "xAttr",
    "xEffect",
    "xIntent",
    "xNeed",
    "xReact",
    "xReason",
    "xWant",
    ]

print("model loading ...")
comet = Comet("./comet-atomic_2020_BART")
comet.model.zero_grad()
print("model loaded")



model loading ...
model loaded


Run your queries. `head` is the input sentence, and `rel` is the relation, as seen in `all_relations`.

In [10]:
# Example code to run COMET. A method has been written for you further down called callCOMET().
queries = []
head = "PersonX relies on PersonY"
rel = "xNeed"
query = "{} {} [GEN]".format(head, rel)
queries.append(query)
print(queries)
results = comet.generate(queries, decode_method="beam", num_generate=5)
print(results)

['PersonX relies on PersonY xNeed [GEN]']
[[' to ask PersonY for help', ' to be dependent on PersonY', ' to ask PersonY to help', ' to know PersonY', ' to ask for help']]


In [11]:
"""
# Optional: Get ATOMIC data (if you wanted the train/test/val sets)
!wget https://ai2-atomic.s3-us-west-2.amazonaws.com/data/atomic2020_data-feb2021.zip
!unzip atomic2020_data-feb2021.zip
"""

'\n# Optional: Get ATOMIC data (if you wanted the train/test/val sets)\n!wget https://ai2-atomic.s3-us-west-2.amazonaws.com/data/atomic2020_data-feb2021.zip\n!unzip atomic2020_data-feb2021.zip\n'

# Parse the sentence to feed into COMET
You're welcome to change this to fit your needs

In [12]:
import nltk
from nltk.tree import Tree
from collections import Counter, defaultdict


class SentParser:
  """
  Parse the sentence and get the entities and the new phrase with tags
  """
  def __init__(self, sentence):
    sentence = sentence.replace(".","")
    parse = nlp(sentence) # call Stanza
    self.phrase = sentence # original sentence
    self.entities = dict() # dict of labels for entities e.g., [{PersonX: John}]
    self.new_phrase = sentence # the sentence with person names changed to PersonX and PersonY
    
    for sentence in parse.sentences:
      ents = self.getEntities(sentence.tokens)
      self.entities = ents

      for tag in ents.keys():
        person = ents[tag]
        self.new_phrase = self.new_phrase.replace(person, tag)
        
  def getEntities(self, parse):
    """
    get the named entities so you can pass it to ATOMIC's input and
    fill the PersonX and PersonY tags from the output

    args:
    parse (list) - list of Stanza token objects for this phrase

    return:
    entities (dict) - keeps track of who is PersonX and PersonY e.g. {PersonX: John}
    """
    entities = dict()
    count = 0
    for word in parse:
      if "PERSON" in word.ner:
        if count == 0:
          entities['PersonX'] = word.text 
          count+=1
        elif count == 1:
          entities['PersonY'] = word.text
    return entities


In [13]:
# Example call
s = SentParser("John went to the bank.")
print(s.phrase)
print(s.entities)
print(s.new_phrase)

John went to the bank
{'PersonX': 'John'}
PersonX went to the bank


# TODO: Setup your schema

You're welcome to use all or just a subset of the relations from `all_relations`. Work with whatever you think makes sense.

Tip: You might want to not take every single fact that ATOMIC gives you. Try to come up with a heuristic to just take what you need.

In [14]:
precondition_relations = ['xIntent', 
                          'xNeed', 
                          'HasPrerequisite', 
                          'xAttr', 
                          'isAfter',
                          'AtLocation', 
                          'LocatedNear', 
                          'LocationOfAction', 
                          'MadeUpOf', 
                          'PartOf',
                          'UsedFor', 
                          'ObjectUse',
                          'HasProperty', 
                          'CapableOf',
                          'Desires', 
                          ]

not_precondition_relations = ['HinderedBy',
                              'NotCapableOf', 
                              'NotDesires',
                              'NotHasProperty',
                              'NotMadeOf']

effect_relations = ['xReact',
                    'xWant',
                    'xReason',
                    'xEffect',
                    'oReact',
                    'oWant',
                    'oEffect',
                    'isBefore',
                    'Causes']

In [15]:
!pip install -U -q sentence-transformers

     |████████████████████████████████| 79 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 47.7 MB/s 


In [16]:
## Import and test S-BERT
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939


In [17]:
def callCOMET(sent, rel, decode="beam", num=5):
  """
  Making COMET generate facts based on an input sent and a relation rel
  You can also provide the decoding method and the number of facts 
  you want it to output.
  """
  query = ["{} {} [GEN]".format(sent, rel)]
  gen = comet.generate(query, decode_method=decode, num_generate=num)[0]
  return [s.strip().replace(".","") for s in gen if s.strip() != "none" or s.strip() != "."]

def fillEntityTags(fact, NER):
  """
  Given a output fact from COMET (str) and the NER (dict), replace the PersonX/Y
  tags with their original names
  """
  new = fact
  for entity in NER.keys():
    new = new.replace(entity, NER[entity])
  return new

class Predicate:
  """
  Individual precondition and effect objects
  """
  def __init__(self, rel, statement, isPre, neg=False):
    self.relation = rel # string - input COMET relation
    self.statement = statement # string - results from COMET
    self.isPrecondition = isPre # boolean - if it's a precondition (True) or an effect (False)
    self.isNegated = neg # boolean - if it's negated (True) or not (False)

class Schema:
  """
  Your schema
  """
  def __init__(self, starting_state):
    self.state = starting_state # defaultdict(set) - a set of facts for each entity
    self.curr_event = "" # string - received from the parser; phrase from the original input sentence with PersonX/PersonY labels
    self.curr_NER = dict() # dict - person names and their corresponding tags for a given subevent/phrase
    self.preconditions = [] # list - stores Predicate objects
    self.timestep = 0 # int - how far you are in the story (optional)

  ### Check the preconditions against the state ###
  def checkPrecondition(self, pred):
    """
    Given this precondition and the current state, does this precondition pass?
    args:
    pred (Predicate)
    
    return:
    boolean - whether or not this event is valid
    """
    # print("--- checkPrecondition ---")
    if pred.isPrecondition == False: return False #it's an effect, don't consider it

    ### Your code here ###
    #TODO: check pred against self.state
    # You can do direct matches or "close enough" (i.e., similarity) matches
    #e.g.:
    # print("pred", pred)
    # print("pred.statement", pred.relation, pred.statement)

#     #Compute embedding for both lists
    
    precondition_list = [pred.statement] * len(self.state[self.curr_NER['PersonX']])
    state_list = list(self.state[self.curr_NER['PersonX']])

    embeddings_precondition = model.encode(precondition_list, convert_to_tensor=True)
    embeddings_state = model.encode(state_list, convert_to_tensor=True)

    cosine_scores = util.cos_sim(embeddings_precondition, embeddings_state)

    
    res = torch.any(cosine_scores > 0.7)
    # res = True in (ele > 0.7 for ele in cosine_scores)

    return res

    # if any(cosine_scores)

    # if 'PersonX' in self.curr_NER and pred.statement not in self.state[self.curr_NER['PersonX']]:
    #   return False
    # ######################
    # if 'PersonX' not in self.curr_NER:
    #     print("self.curr_NER", self.curr_NER)
    #     return True

    # return True

  def getPreconditions(self):
    """
    Given the input event string (self.curr_event), 
    return a list of preconditions (list of Predicate objects)
    """
    # print("--- getPreconditions ---")
    pre = []  

    ### Your code here ###
    #TODO: complete the list using relations from "all_relations"
    #an example:

    for rel in precondition_relations:
        comet_out = callCOMET(self.curr_event,rel)
        for fact in comet_out:      
            filled = fillEntityTags(fact,self.curr_NER)
            pre.append(Predicate(rel, filled, True)) #Predicate(rel, statement, isPrecondition, negated)
    
    # maybe need to change to check the opposite of relation
    for rel in not_precondition_relations:
        comet_out = callCOMET(self.curr_event,rel)
        for fact in comet_out:      
            filled = fillEntityTags(fact,self.curr_NER)
            pre.append(Predicate(rel, filled, False)) #Predicate(rel, statement, isPrecondition, negated)
    ######################

    return pre    

  def checkValidity(self):
    """
    Goes through all the preconditions to check to see if
    this event can be added to the state.
    A precondition is considered valid as long as it matches
    directly or "close enough" to a fact already in the state.
    
    return:
    boolean - whether or not this event is valid
    """
    # print("--- checkValidity ---")
    print("state", self.state)
    preconds = self.getPreconditions()
    # print("preconds", preconds)
    valids = []
    for precond in preconds:
      print("precond.statement", precond.relation, precond.statement)
      valid = self.checkPrecondition(precond)
      print("valid", '✓' if valid else '✗')
      if valid:
        valids.append(precond)
    if valids:
      self.preconditions += valids
      return True    
    # print("self.preconditions", self.preconditions)
    return False


  ### Once validated, update the schema ###
  def getEffects(self):
    """
    Given the input event string (self.curr_event),
    return a list of effects (list of Predicate objects)
    """
    # print("--- getEffects ---")
    effects = []

    ### Your code here ###
    #TODO: complete the list using relations from "all_relations"
    #an example:
    # rel = "xReact"
    for rel in effect_relations:
        fact = callCOMET(self.curr_event,rel)[0]
        filled = fillEntityTags(fact, self.curr_NER)
        effects.append(Predicate(rel, filled, False)) #Predicate(rel, statement, isPrecondition, negated)
        # print("fact", fact, "\t filled", filled)
    ######################

    return effects   


  def updateSchema(self, event, NER_dict):
    """
    Given an input event string (event), check the validity of adding it to the state,
    and update the schema state (self.state) with new effects
    args:
    event(str) - received from the parser; phrase from the original input sentence with PersonX/PersonY labels
    NER_dict(dict) - person names and their corresponding tags for a given subevent/phrase, e.g. NER_dict['PersonX'] = "Cindy"
    """
    # print("=== updateSchema ===")
    # print("event", event)
    # print("NER_dict", NER_dict)
    self.curr_event = event
    self.curr_NER = NER_dict

    #1) check validity
    valid = self.checkValidity()
    if len(self.preconditions) > 0:
        print("Preconditions",self.preconditions[0].statement)
    else:
        print("no procondition")

    #2) update the state
    effects = self.getEffects()
    if len(effects) > 0:
        print("Effects", effects[0].statement)
    else:
        print("no effects")



    ### Your code here ###

    #TODO: add these new effects to the state    
    #   if effect.relation == "xReact":
    #     if 'PersonX' in NER_dict:
    #         self.state[NER_dict['PersonX']].add(effect.statement)
    #     else:
    #         print("personx not in ner_dict", NER_dict)
    for effect in effects:
        for effect.relation in effect_relations:
            self.state[NER_dict['PersonX']].add(effect.statement)

    # TODO: (optional) remove facts that aren't true anymore
    # Although this is ideal, it might be hard to figure out when facts are negated

    ######################





# What to Turn In
* Your code
  * Including your answers outputted for the Story Tracking Questions
* A document, including:
  * A brief description/diagram of the structure of your schema
  * Your answers to the questions about COMET-ATOMIC




## Story Tracking Questions
Run the following stories through your system and print out your schema after each sentence (or subevent if there are multiple events in a sentence).

For each scenario, report:
- the number of the scenario,
- the bit of code that runs the story, and
- keep the output of your schema on that story in your ipynb. It should be printing the schema at each timestep. (No fancy formatting is necessary. Just printing the dictionary is fine.)

Note: The first sentence of each story should initialize your schema state.

**Do not change your schema code in between running these examples! Your final schema code should be able to run multiple scenarios.**

In [18]:
stories = {
    1: ["Gina misplaced her phone.", "Gina looks for her phone in the living room.", "Gina remembers leaving her phone in the car.", "Gina goes back to the car.", "Gina finds her phone in the car."],
    2: ["Phil was at the community pool.","Phil thought he could go out to the deeper end by himself.","Phil jumps into the deep end.","Phil has trouble staying afloat.","The lifeguard had to help Phil out of the water."],
    3: ["Amy was happy her first class in junior high was all new kids.", "Amy introduced herself to the girl seated next to her.", "The girl was even more nervous than Amy to make friends.", "The girls talked and bonded over their love of books.", "The girls decided to meet up after school to go to the library."],
    4: ["Xander's dog hates his treats.", "Xander decided to go buy some new dog treats.", "None of the dog treats at the pet store looked tasty.", "Xander decided to buy his dog some salmon from the fish market.", "Xander's dog loved the salmon."],
    5: ["Tim has never cooked for his family.", "Tim decided to follow an old family recipe.", "Tim's grandma told him anybody could make the recipe.", "Tim made a whole meal for his family in one hour.", "Tim's family all loved the meal."]
}

In [19]:
# This is where everything is called
start = defaultdict(set)
start.update({"John": set(["John is hungry"])})

# Example call
schema = Schema(start)
sent = "John eats an apple."
s = SentParser(sent)
schema.updateSchema(s.new_phrase, s.entities)
print(schema.state)

state defaultdict(<class 'set'>, {'John': {'John is hungry'}})
precond.statement xIntent to satisfy his hunger
valid ✗
precond.statement xIntent to satisfy their hunger
valid ✗
precond.statement xIntent to be healthy
valid ✗
precond.statement xIntent to eat something healthy
valid ✗
precond.statement xIntent to satisfy hunger
valid ✗
precond.statement xNeed to buy an apple
valid ✗
precond.statement xNeed to get an apple
valid ✗
precond.statement xNeed to pick one up
valid ✗
precond.statement xNeed to pick one
valid ✗
precond.statement xNeed none
valid ✗
precond.statement HasPrerequisite John eats an apple
valid ✗
precond.statement HasPrerequisite John eats the apple
valid ✗
precond.statement HasPrerequisite John takes a bite
valid ✓
precond.statement HasPrerequisite John is hungry
valid ✓
precond.statement HasPrerequisite none
valid ✗
precond.statement xAttr hungry
valid ✗
precond.statement xAttr healthy
valid ✗
precond.statement xAttr satiated
valid ✗
precond.statement xAttr hungry
va

In [20]:
def generate_story_schema(story):
    # Running through the above stories
    start = defaultdict(set)
    schema = Schema(start)
    person = SentParser(story[0]).entities['PersonX'] # there's no PersonY in this HW 
    start[person] = set([story[0]]) # initalize with the first sentence
    success_list = [1] # assume the first sentence is already initialized successsfully
    for idx, sent in enumerate(story[1:]):
        try: 
            s = SentParser(sent)
            schema.updateSchema(s.new_phrase,s.entities)
            success_list.append(idx+2) # 1-indexed and begin at the 2nd sentence
        except:
            print(f"Error. Skip this sentence: {sent}")
            continue
        print(schema.state)
    print(f"Success on {len(success_list)}/{len(story)} sentences with index {tuple(success_list)} (1-indexed)")

In [21]:
story_id = 1
generate_story_schema(stories[story_id])

state defaultdict(<class 'set'>, {'Gina': {'Gina misplaced her phone.'}})
precond.statement xIntent to find her phone
valid ✗
precond.statement xIntent to find their phone
valid ✗
precond.statement xIntent to find his phone
valid ✗
precond.statement xIntent to find the phone
valid ✗
precond.statement xIntent to find it
valid ✗
precond.statement xNeed to have lost their phone
valid ✗
precond.statement xNeed to be in the house
valid ✗
precond.statement xNeed to look around the room
valid ✗
precond.statement xNeed to look for it
valid ✗
precond.statement xNeed to look around
valid ✗
precond.statement HasPrerequisite Gina looks for her phone
valid ✓
precond.statement HasPrerequisite Gina looks for the phone
valid ✓
precond.statement HasPrerequisite Gina looks for their phone
valid ✓
precond.statement HasPrerequisite Gina finds her phone
valid ✓
precond.statement HasPrerequisite Gina finds the phone
valid ✓
precond.statement xAttr curious
valid ✗
precond.statement xAttr lost
valid ✗
precond

In [22]:
story_id = 2
generate_story_schema(stories[story_id])

state defaultdict(<class 'set'>, {'Phil': {'Phil was at the community pool.'}})
precond.statement xIntent to be alone
valid ✗
precond.statement xIntent to have fun
valid ✗
precond.statement xIntent to be independent
valid ✗
precond.statement xIntent to be adventurous
valid ✗
precond.statement xIntent to explore
valid ✗
precond.statement xNeed to get in the water
valid ✗
precond.statement xNeed to go to the pool
valid ✗
precond.statement xNeed to go to the water
valid ✗
precond.statement xNeed to think about it
valid ✗
precond.statement xNeed to have a plan
valid ✗
precond.statement HasPrerequisite Phil jumps into the water
valid ✗
precond.statement HasPrerequisite Phil jumps in the water
valid ✗
precond.statement HasPrerequisite Phil jumps in the pool
valid ✓
precond.statement HasPrerequisite Phil jumps into the pool
valid ✓
precond.statement HasPrerequisite Phil goes swimming
valid ✗
precond.statement xAttr adventurous
valid ✗
precond.statement xAttr brave
valid ✗
precond.statement xA

In [23]:
story_id = 3
generate_story_schema(stories[story_id]) # SentParser cannot deal with the PersonX in the last two sentences 

state defaultdict(<class 'set'>, {'Amy': {'Amy was happy her first class in junior high was all new kids.'}})
precond.statement xIntent to be friendly
valid ✗
precond.statement xIntent to be friendly
valid ✗
precond.statement xIntent to be social
valid ✗
precond.statement xIntent to make friends
valid ✗
precond.statement xIntent to meet someone
valid ✗
precond.statement xNeed to find the girl
valid ✗
precond.statement xNeed to find a seat
valid ✗
precond.statement xNeed to sit down
valid ✗
precond.statement xNeed to get up
valid ✗
precond.statement xNeed none
valid ✗
precond.statement HasPrerequisite Amy sits next to the girl
valid ✗
precond.statement HasPrerequisite Amy sits next to a girl
valid ✗
precond.statement HasPrerequisite Amy asks the girl to dance
valid ✗
precond.statement HasPrerequisite Amy makes a new friend
valid ✗
precond.statement HasPrerequisite Amy sits down
valid ✗
precond.statement xAttr friendly
valid ✗
precond.statement xAttr sociable
valid ✗
precond.statement xA

In [24]:
story_id = 4
generate_story_schema(stories[story_id])

state defaultdict(<class 'set'>, {'Xander': {"Xander's dog hates his treats."}})
precond.statement xIntent to feed the dog
valid ✗
precond.statement xIntent to feed the dog
valid ✗
precond.statement xIntent to feed his dog
valid ✗
precond.statement xIntent to feed his dog
valid ✗
precond.statement xIntent to feed their dog
valid ✗
precond.statement xNeed to go to the store
valid ✗
precond.statement xNeed to have a dog
valid ✗
precond.statement xNeed to go to a store
valid ✗
precond.statement xNeed to have a pet
valid ✗
precond.statement xNeed to have money
valid ✗
precond.statement HasPrerequisite Xander goes to the store
valid ✗
precond.statement HasPrerequisite Xander feeds the dog treats
valid ✓
precond.statement HasPrerequisite Xander buys dog treats
valid ✓
precond.statement HasPrerequisite Xander feeds the dog
valid ✓
precond.statement HasPrerequisite none
valid ✗
precond.statement xAttr responsible
valid ✗
precond.statement xAttr hungry
valid ✗
precond.statement xAttr determined

In [25]:
story_id = 5
generate_story_schema(stories[story_id])

state defaultdict(<class 'set'>, {'Tim': {'Tim has never cooked for his family.'}})
precond.statement xIntent to be a good cook
valid ✗
precond.statement xIntent to make a good meal
valid ✗
precond.statement xIntent to follow a family recipe
valid ✗
precond.statement xIntent to follow the family recipe
valid ✗
precond.statement xIntent to follow a family tradition
valid ✗
precond.statement xNeed to read the recipe
valid ✗
precond.statement xNeed to learn the recipe
valid ✗
precond.statement xNeed to know the recipe
valid ✗
precond.statement xNeed to have a recipe
valid ✗
precond.statement xNeed none
valid ✗
precond.statement HasPrerequisite Tim eats the recipe
valid ✗
precond.statement HasPrerequisite Tim cooks a meal
valid ✗
precond.statement HasPrerequisite Tim makes a recipe
valid ✗
precond.statement HasPrerequisite Tim makes a cake
valid ✗
precond.statement HasPrerequisite none
valid ✗
precond.statement xAttr determined
valid ✗
precond.statement xAttr creative
valid ✗
precond.state

## Questions about COMET-ATOMIC



1.   What types of stories is COMET-ATOMIC *good* at tracking? In other words, what types of information is it good at modeling? (It might help to think about how COMET-ATOMIC compares to other knowledge bases.)

COMET-ATOMIC is generally good at tracking: (1) stories where the steps are connected closely so the gap wouldn't be too large to apply the if-else relationship in the knowledge graph (2) stories that have ordinary background and common-sense-based otherwise it would be out of the scope of the knowledge graph

2.   What types of stories is COMET-ATOMIC *bad* at tracking?

COMET-ATOMIC is generally bad at tracking: (1) stories that jump too fast or lack logic. (2) stories that have a very complicated narrative or alien background. Also, since the given sentence parser is not good at extracting the pronoun as `PersonX' or `PersonY' so the stories which are full of pronoun-only sentences are not suitable at this system. For example, in the 3rd story, the subject switches from Amy to the girl and the girls, and the parser cannot extract the girls as the subject and referring to Amy and the girl.

   2a. Do you think any of the other knowledge bases mentioned in class could better model these?

   GPT-3 can work as complementary module to address these issues, due to its causal generation ability. In detail, we can fine-tune GPT-3 to generate a more consistent story first given the original sentences of story, then feed to COMET for the schema.



# Grading Rubric
This assignment is 10% of your final grade, broken down as the following for grading:
*   Code - 2 points
*   Schema Diagram/Explanation - 2 points
*   Story Tracking Questions - 15 points
*   COMET-ATOMIC Questions - 6 points

Total: 20 points
